Evaluate global model
Traceback (most recent call last):
  File "C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Personalized_Federated_Learning\main.py", line 271, in \<module>
    run(args)
  File "C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Personalized_Federated_Learning\main.py", line 102, in run
    server.train()
  File "C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Personalized_Federated_Learning\flcore\servers\serveravg.py", line 33, in train
    self.evaluate()
  File "C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Personalized_Federated_Learning\flcore\servers\serverbase.py", line 261, in evaluate
    stats = self.test_metrics()
  File "C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Personalized_Federated_Learning\flcore\servers\serverbase.py", line 231, in test_metrics
    ct, ns, auc = c.test_metrics()
  File "C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Personalized_Federated_Learning\flcore\clients\clientbase.py", line 173, in test_metrics
    test_acc += (torch.sum(torch.argmax(output, dim=1) == y)).item()
RuntimeError: The size of tensor a (1200) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
def test_metrics(self):
    testloaderfull = self.load_test_data()
    simulate_data_streaming(testloaderfull)
    # self.model = self.load_model('model')
    # self.model.to(self.device)
    self.model.eval()

    test_acc = 0
    test_num = self.F.size()[1]
    #y_prob = []
    #y_true = []

    with torch.no_grad():
        test_acc = CPHSLoss(self.F, self.model.weight, self.V, test_num, lambdaF=self.lambdaF, lambdaD=self.lambdaD, lambdaE=self.lambdaE, Nd=2, Ne=self.pca_channels, return_cost_func_comps=False)
        
    # self.model.cpu()
    # self.save_model(self.model, 'model')

    #y_prob = np.concatenate(y_prob, axis=0)
    #y_true = np.concatenate(y_true, axis=0)
    # AUC not defined for regression, only for classification
    #auc = metrics.roc_auc_score(y_true, y_prob, average='micro')
    
    return test_acc, test_num